In [48]:
from urllib import *
from html import unescape
import requests
import json 
headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}
import requests
import time
import bs4
from bs4 import BeautifulSoup
import re
import time


def download(method, url, 
             param = None, data = None, 
            timeout = 1,maxretries = 3):
    
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}
    #headers = None
    
    try:
        resp = requests.request(method, url, params= param, data=data, headers=headers)
        resp.raise_for_status()
        
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries > 0: 
            time.sleep(timeout) # param에따라 몇초 기다릴지 결정 할 수가 있다.
            print(maxretries) # 재귀적으로 자기 자신을 부르게 코드를 짜면 된다.
            resp = download(method,
                            url, param = param, data=data,
                            timeout=timeout,
                            maxretries = maxretries - 1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

def checkBlog(url):
    return requests.compat.urlparse(url)[1].endswith("tistory.com")
    ## 정규식 re.search(".+?\.tistory.com", "http:\\adam24eve.tistory.com/") #짧게매칭되는거 ? 레이지 

def parseURL(seed):
    html = download("get",seed)
    dom = BeautifulSoup(html.text, "lxml")
    
    ## naver iframe html안에 html 또들어있어서
   # if len(dom.select("#mainFrame")) < 1:  ## 메인이랑, 첫페이지만 없음
   #     return []
    
   
   # seed = requests.compat.urljoin(seed,dom.select("#mainFrame")[0]["src"])
  #  print("-=================================================")
   # print(seed)
  #  print("-=================================================")
   # html = download("get",seed)
   # dom = BeautifulSoup(html.text, "lxml")
    

    return  [requests.compat.urljoin(seed, _["href"]) for _ in dom.find_all("a") if  _.has_attr("href") and len(_["href"]) >3 and checkBlog(requests.compat.urljoin(seed, _["href"]))]




In [49]:
url = "https://search.daum.net/search?w=tot&DA=YZR&t__nil_searchbox=btn&sug=&sugo=&q=%EB%B0%95%EB%B3%B4%EC%98%81"
html = download("get",url)
dom = BeautifulSoup(html.text, "lxml")

In [50]:
 requests.compat.urlparse(url)

ParseResult(scheme='https', netloc='search.daum.net', path='/search', params='', query='w=tot&DA=YZR&t__nil_searchbox=btn&sug=&sugo=&q=%EB%B0%95%EB%B3%B4%EC%98%81', fragment='')

In [51]:
dom

<!DOCTYPE html>
<html class="win chrome " lang="ko" xmlns="http://www.w3.org/1999/xhtml">
<head profile="http://a9.com/-/spec/opensearch/1.1/">
<meta content="text/html;charset=utf-8" http-equiv="content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="off" name="autocomplete"/>
<meta content="always" name="referrer"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="박보영 – Daum 검색" property="og:title"/>
<meta content="https://search.daum.net/search?w=tot&amp;q=%EB%B0%95%EB%B3%B4%EC%98%81" property="og:url"/>
<meta content="Daum 검색에서 박보영에 대한 최신정보를 찾아보세요." property="og:description"/>
<meta content="https://search1.daumcdn.net/search/statics/common/img/og_search.png" property="og:image"/>
<meta content="다음검색" property="og:site_name"/>
<title>박보영 – Daum 검색</title>
<link href="//search.daum.net/OpenSearch.xml" rel="search" title="Daum" type="application/opensearchdescription+xml"/>
<link charset="utf-8" href="//search1.daumcdn.net/search/sta

In [54]:
[_["href"] for _ in dom.select("#blogColl a.f_link_b")if checkBlog(_["href"]) ]

['http://adam24eve.tistory.com/858',
 'http://cscs0717.tistory.com/27',
 'http://listup.tistory.com/248',
 'http://34344541.tistory.com/43']

In [53]:
queue = list() ##하나 생김
queue.extend( [_["href"] for _ in dom.select("#blogColl a.f_link_b") if checkBlog(_["href"])])
seen = list()
while queue :
    baseURL = queue.pop(0)  ##지금 너비우선
                            ##pop(0)을 깊이우선 -1로 바꿔서 depth 를 정해놓고 or 특정도메인내에서만 => 포커스드크롤링
    print(baseURL)
    seen.append(baseURL)
    # 슬립 
    time.sleep(1)
    linkList = parseURL(baseURL) # 중복된 url 계속 생김 re??
 
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))
  ##  queue.extend(linkList)
    
    #새로운 url queue 보고알고, 얼마나 방문했는지 Seen q보면암
    ## 큐의 사이즈 무한정으로 늘어남
    ## queue,랑 seen합쳐야 400개정도 나옴


    ##파이썬이니까 코드가 짧다.

http://adam24eve.tistory.com/858
Queue: 44, Seen: 1
http://cscs0717.tistory.com/27
Queue: 60, Seen: 2
http://listup.tistory.com/248
Queue: 83, Seen: 3
http://34344541.tistory.com/43
Queue: 90, Seen: 4
http://adam24eve.tistory.com/category
Queue: 96, Seen: 5
http://adam24eve.tistory.com/category/Cosmos
Queue: 102, Seen: 6
http://adam24eve.tistory.com/category/Mystery%20world
Queue: 108, Seen: 7
http://adam24eve.tistory.com/category/Star
Queue: 111, Seen: 8
http://adam24eve.tistory.com/category/Political%20
Queue: 118, Seen: 9
http://adam24eve.tistory.com/category/It-game
Queue: 121, Seen: 10
http://adam24eve.tistory.com/guestbook
Queue: 120, Seen: 11
https://adam24eve.tistory.com/rss


KeyboardInterrupt: 

In [77]:
python --version

NameError: name 'python' is not defined

In [ ]:
### 링크한 2뎁스 까지만
## 특정 도메인에서 2~3뎁스까지만 수집 
## 정보검색에서 쓰인다.

In [70]:
def parseURL(seed):
    html = download("get",seed[0])
    dom = BeautifulSoup(html.text, "lxml")
    
    ## naver iframe html안에 html 또들어있어서
   # if len(dom.select("#mainFrame")) < 1:  ## 메인이랑, 첫페이지만 없음
   #     return []
    
   
   # seed = requests.compat.urljoin(seed,dom.select("#mainFrame")[0]["src"])
  #  print("-=================================================")
   # print(seed)
  #  print("-=================================================")
   # html = download("get",seed)
   # dom = BeautifulSoup(html.text, "lxml")
    

    return  [(requests.compat.urljoin(seed[0], _["href"]),seed[1]+1)
             for _ in dom.find_all("a") 
             if seed[1] < 2 and  _.has_attr("href") and len(_["href"]) >3 and checkBlog(requests.compat.urljoin(seed[0], _["href"]))]
# 뎁스를 2까지만 하겠다.



In [76]:
queue = list() ##하나 생김
queue.extend( [(_["href"],1) for _ in dom.select("#blogColl a.f_link_b") if checkBlog(_["href"])])
seen = list()
while queue :
    baseURL = queue.pop(0)  ##지금 너비우선
                            ##pop(0)을 깊이우선 -1로 바꿔서 depth 를 정해놓고 or 특정도메인내에서만 => 포커스드크롤링
    print(baseURL)
    seen.append(baseURL)
    # 슬립 
    #time.sleep(1)
    linkList = parseURL(baseURL) # 중복된 url 계속 생김 re??
 
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))
  ##  queue.extend(linkList)
    
    #새로운 url queue 보고알고, 얼마나 방문했는지 Seen q보면암
    ## 큐의 사이즈 무한정으로 늘어남
    ## queue,랑 seen합쳐야 400개정도 나옴


    ##파이썬이니까 코드가 짧다.

('http://adam24eve.tistory.com/858', 1)
Queue: 45, Seen: 1
('http://cscs0717.tistory.com/27', 1)
Queue: 61, Seen: 2
('http://listup.tistory.com/248', 1)
Queue: 85, Seen: 3
('http://34344541.tistory.com/43', 1)
Queue: 92, Seen: 4
('http://adam24eve.tistory.com/category', 2)
Queue: 91, Seen: 5
('http://adam24eve.tistory.com/category/Cosmos', 2)
Queue: 90, Seen: 6
('http://adam24eve.tistory.com/category/Mystery%20world', 2)
Queue: 89, Seen: 7
('http://adam24eve.tistory.com/category/Star', 2)
Queue: 88, Seen: 8
('http://adam24eve.tistory.com/category/Political%20', 2)
Queue: 87, Seen: 9
('http://adam24eve.tistory.com/category/It-game', 2)
Queue: 86, Seen: 10
('http://adam24eve.tistory.com/guestbook', 2)
Queue: 85, Seen: 11
('https://adam24eve.tistory.com/rss', 2)
Queue: 84, Seen: 12
('http://adam24eve.tistory.com/858', 2)
Queue: 83, Seen: 13
('http://adam24eve.tistory.com/tag/%EA%B9%80%EC%98%81%EA%B4%91', 2)
Queue: 82, Seen: 14
('http://adam24eve.tistory.com/tag/%EA%B9%80%EC%98%81%EA%B4%91

In [ ]:
##96번

In [ ]:
#seen 뿌리면 어떤 사이트인지 뿌려짐
## 큐 메모레 적당선에서 끊어줘야함
## => 디비통해서 효율적 관리

In [ ]:
## 크롤링 네트워크 타고다니면서 링크 어떻게 뻗어나가있는지, 들어오는링크 나가는 링크 분석하는 도구
## 스크래핑은 뽑고싶은 데이터 어디에 있는지 뽑아서 db에 저장

## => 웹수집기, 크롤러

In [82]:
version

NameError: name 'version' is not defined